In [2]:
import pandas as pd
import tqdm
import sys
import os

%load_ext autoreload
%autoreload 2
sys.path.append("/Users/xinc./Documents/GitHub/note")    # for mac
from module.get_info_Postgre import PostgresClient

# read

In [ ]:
# 讀取資料庫中的 disposal_info 表格
database_name = "disposal"
client = PostgresClient(database = database_name)
disposal_info = client.fetch_table(table = "disposal_info")